In [1]:
import sympy as sp
import minterpy as mp
import numpy as np
from minterpy.pointcloud_utils import *

#from mpl_toolkits import mplot3d
%matplotlib inline

#from mpl_toolkits.mplot3d import axes3d

import torch
#import torchvision
#from torchvision import transforms, datasets

import random
import numpy as np
#import matplotlib.pyplot as plt

#import os
#from operator import itemgetter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#import ot

#from sklearn.neighbors import NearestNeighbors

#import ripser
#import persim
#from persim import plot_diagrams

from operator import itemgetter


#do pip installs as follows to use vedo for plotting point clouds
#pip install vedo
#pip install ipyvtklink

#import numpy as np
from vedo import *

#import matplotlib.pyplot
#from mpl_toolkits.mplot3d import Axes3D

In [2]:
def give_centeroid(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return np.array([[sum_x/length, sum_y/length, sum_z/length]])

In [3]:
def give_next_neighbours_barycenter_indices(batch_x, input_barycenter, remaining_indices, sweep_radsius):
    
  #sweep_radsius = 0.2
  #num_neighbours = int(batch_x.shape[0] / no_of_barycenetrs_required)
    
  wasserDistance = []
  distance_cum_index = np.array([])

  for j in remaining_indices:
    
    wassDistance = dist = np.linalg.norm(batch_x[j]-input_barycenter)

    distance_cum_index = np.concatenate((distance_cum_index, np.array([wassDistance, j])), axis = 0)
   
  distance_cum_index = distance_cum_index.reshape(int(distance_cum_index.shape[0]/2), 2)
  distance_cum_index = sorted(distance_cum_index, key=itemgetter(0))
  
  distance_cum_index = np.array(distance_cum_index)
  
  only_distances = distance_cum_index[:,0]
  #print(only_distances)
  where_is_it = np.where( only_distances < sweep_radsius ) 
  #print('where_is_it', where_is_it)  
  #print('where_is_it[0][-1]', where_is_it[0][-1])
  num_neighbours = where_is_it[0][-1] +1
  #print('where_is_it[0]', where_is_it[0])
  #print('where_is_it[0][-1]', where_is_it[0][-1])
  #print('num_neighbours',num_neighbours)
  #break
  #print('distance_cum_index',distance_cum_index[:,1])
  remaining_indices = distance_cum_index[:,1]

  remaining_indices = remaining_indices.astype(int)
  
    
  A = np.array([])
  for i in range(num_neighbours):
    if(i >= distance_cum_index[:,1].shape[0]):
        break
    A = np.concatenate((A, batch_x[int(distance_cum_index[:,1][i])]), axis = 0 )
  
  A = A.reshape(int(A.shape[0]/3) , 3)
  
  #print("The shape of A is ")
  #print(A.shape)
  next_barycenter = give_centeroid(A) 
  
  next_barycenter = np.array(next_barycenter)
  #print(next_barycenter.shape)
  next_barycenter = next_barycenter.reshape(next_barycenter.shape[0]* next_barycenter.shape[1])
  #print(next_barycenter)  
  return A, next_barycenter, remaining_indices,num_neighbours


In [4]:
def get_convergent_barycenters(point_cloud, initial_pt,sweep_radsius):   
    
    #no_neighbours = int(point_cloud.shape[0] / no_of_barycenetrs_required)
    #initiating no of neighbours
    #no_neighbours = 5
    
    #num_neighbours = int(batch_x.shape[0] / no_of_barycenetrs_required)

    
    bary = initial_pt
    rem_indices = np.array(range(0,point_cloud.shape[0]))
    #print(rem_indices)
    #print("Size of batch : ", point_cloud.shape[0])
    sampled_barycenters = np.array([])
    sampled_barycenters = torch.tensor(sampled_barycenters)
    covered_indices = np.array([])

    for i in range(int(point_cloud.shape[0])):

        if(len(rem_indices) == 2):
            #print("END")
            break

        #print("Iteration number : ", i+1)
        #print("Input barycenter : ")


        old_bary = bary


        #print('rem_indices before', rem_indices)
        neighbours, bary, rem_indices, no_neighbours = give_next_neighbours_barycenter_indices(point_cloud, bary, rem_indices, sweep_radsius)
        #print('no_neighbours', no_neighbours)
        #break
        wassDistance = np.linalg.norm(old_bary - bary)  

        covered_indices = np.concatenate((covered_indices, rem_indices[:4] ) ,axis = 0)


        #print("Tracking distance between new barycenter and previous barycenter : ",wassDistance )
        if(wassDistance < 0.000001):

            unique_covered_indices = np.unique(covered_indices, axis=0)
            sampled_barycenters = torch.cat((sampled_barycenters, torch.tensor(bary)), 0)

            s1 = set(rem_indices)
            s2 = set(unique_covered_indices)
            rem_set = s1 - s2
            rem_inds = list(rem_set)
            #rem_indices = rem_set
            rem_indices = rem_indices[no_neighbours:]

            #print("Sampled barycenters are")
            #print(sampled_barycenters)
            #no_neighbours = 5
            #print('rem_indices',rem_indices)
            #print("len(rem_indices)",len(rem_indices))
            if(len(rem_indices) ==0):
                break
            bary = point_cloud[rem_indices[0]]
    sampled_barycenters = sampled_barycenters.reshape(int((sampled_barycenters.shape[0]/3)),3)
    
    return sampled_barycenters

In [5]:
def _compute_distance_matrix(x, p=2):
    x_flat = x.view(x.size(0), -1)

    distances = torch.norm(x_flat[:, None] - x_flat, dim=2, p=p)

    return distances

In [6]:
def get_persistence_diagram(point_cloud, maximum_dim):

    point_cloud = torch.tensor(point_cloud)

    dist_matrix = _compute_distance_matrix(point_cloud, p=2)
    diagrams = ripser.ripser(dist_matrix.cpu().detach().numpy(), distance_matrix=True, maxdim=maximum_dim)['dgms']
    return diagrams, plot_diagrams(diagrams, show=True)


# Standford bunny

In [7]:
point_data_bunny = np.load('/media/chethan/New Volume/Thesis/Project/barycenter_convergence_3d/bunny_coords.npy')

In [8]:
point_data_bunny.shape

(34817, 3)

In [9]:
x_t = point_data_bunny[:,0]
y_t = point_data_bunny[:,1]
z_t = point_data_bunny[:,2]


plot_points = [x_t,y_t,z_t]
Points(plot_points).show(axes=1)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [10]:
#rand_sampled_standford_bunny = point_data_bunny[np.random.randint(0,34817, 132)]

# barycenter subsampling


In [31]:
sweeing_radius = 0.06

In [32]:
bunny_bary = get_convergent_barycenters(point_data_bunny, point_data_bunny[0],sweeing_radius) #started at 17.48 0.06

In [33]:
bunny_bary.shape

torch.Size([3089, 3])

In [34]:
#np.save('/media/chethan/New Volume/Thesis/Project/barycenter_convergence_3d/bunny_subsampled_3089_points.npy', bunny_bary)

In [35]:
#bunny_bary = np.load('/media/chethan/New Volume/Thesis/Project/barycenter_convergence_3d/bunny_subsampled_1227_points.npy')

In [36]:
# plotting the sampled barycenters

x_stb = bunny_bary[:,0]#.cpu().detach().numpy()

y_stb = bunny_bary[:,1]#.cpu().detach().numpy()

z_stb = bunny_bary[:,2]#.cpu().detach().numpy()



plot_points = [x_stb,y_stb,z_stb]
Points(plot_points).show(axes=1)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

# Random sampling same number of points

In [37]:
rand_sampled_standford_bunny = point_data_bunny[np.random.randint(0,point_data_bunny.shape[0], bunny_bary.shape[0])]

In [38]:
# plotting the sampled barycenters

x_stb = rand_sampled_standford_bunny[:,0]#.cpu().detach().numpy()

y_stb = rand_sampled_standford_bunny[:,1]#.cpu().detach().numpy()

z_stb = rand_sampled_standford_bunny[:,2]#.cpu().detach().numpy()



plot_points = [x_stb,y_stb,z_stb]
Points(plot_points).show(axes=1)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)